# Record Rename

Get rid of the `record` dimension in the RRTMGP flux netCDF files and replace it with `forcing` (working title) since the dimension represents different forcing scenarios.

The only non-standard library is `xarray`, which the user must install in their own Python environment. `PIPPATH` is the assumed location.

In [4]:
import os, sys

# directory in which libraries installed with conda are saved
PIPPATH = '{}/.local/'.format(os.path.expanduser('~')) + \
    'cori/3.7-anaconda-2019.10/lib/python3.7/site-packages'
sys.path.append(PIPPATH)

import xarray as xa

## Paths

The user should provide an existing RRTMGP (or LBLRTM) netCDF with fluxes and heating rates in it. This is different from the netCDF from which RRTMGP reads absorption coefficients.

A name for the data with a `forcing` (working title) dimension should also be provided, but it is not necessary. In any case, the modified dataset is written to a temporary 

In [41]:
inNC = 'temp-lbl.nc'
assert os.path.exists(inNC), 'Could not find {}'.format(inNC)

overwrite = True

outNC = 'lblrtm-lw-flux-inputs-outputs-forcing.nc'

# make sure we don't keep appending to an older file
buffNC = 'temp.nc'
if os.path.exists(buffNC): os.remove(buffNC)

## Dimension Modification

We will open up `inNC` and iterate over all of its variables. If a given variable has a `record` dimension, we will rename it to `forcing` (working title). If not, the variable is retained as is. Then we write to the new netCDF. If `overwrite` is set to `True`, the input netCDF is replaced.

In [42]:
with xa.open_dataset(inNC) as inObj:
    ncVars = list(inObj.keys())

    for ncVar in ncVars: 
        ncDat = inObj[ncVar]

        # append to netCDF if it already exists, start the file if not
        modeNC = 'a' if os.path.exists(buffNC) else 'w'

        if 'record' in ncDat.dims:
            # which dimension corresponds to `record`?
            dims = list(ncDat.dims)
            iRec = dims.index('record')
            dims[iRec] = 'forcing'

            # save variable with new dimensions
            newDat = xa.DataArray(ncDat, dims=dims)
        else:
            # retain any variables without a record dimension
            newDat = xa.DataArray(ncDat)
        # endif record

        newDat.to_netcdf(buffNC, mode=modeNC)
    # end ncVar loop
# endwith

if overwrite: outNC = str(inNC)
    
os.rename(buffNC, outNC)
print('Completed {}'.format(outNC))

Completed temp-lbl.nc
